# Исследование векторного представления элементов и настройка параметров

## Корпус справочных элементов и предобработка

Корпус справочных элементов представляет собой набор из 264 тысяч записей справочника "Номенклатуры"

Минимальная предобработка выполнена с помошью библиотекой Python - NLTK и включает в себя следующую последовательность действий:
* Удаление пунктуации и знаков
* Удаление числовых значений
* Приведение слов к нижнему регистру
* Токенизация

In [5]:
from nltk.tokenize import RegexpTokenizer
import re
import pandas as pd
from tqdm import tqdm as tq

data_file = pd.read_csv('./material.csv', sep=';', encoding = 'cp1251', error_bad_lines=False,
                        low_memory=False)[['FullName']]

data_file = data_file[['FullName']].astype('str')

tokenizer = RegexpTokenizer(r'\w+')

rows = [row[0] for row in data_file.values]

words = []
for row in tq(rows):
    words.extend([i.lower() for i in tokenizer.tokenize(re.sub(r'\d+', '', row)) if len(i) > 1])

100%|███████████████████████████████| 263740/263740 [00:08<00:00, 30656.79it/s]


## Набор гиперпараметров

Параметры | Значения
----------| --------
Размерность вектора | 25 / 50 / **100** / 200 / 400 / 800
Окно | 1 / 2 / 3 / 4 / **5** / 6 / 7 / 8
Минимальная частота | 0 / **5** / 10 / 20 / 50 / 100 / 200/400 / 800 / 1000 / 1200 / 2400
Негативное семплирование | 1 / 2 / 3 / **5** / 8 /10 / 15
Шаг обучения | 0.0125 / **0.025** / 0.05 / 0.1
Сэмплирование | 0 / 1e-1 / 1e-2 / **1e-3** / 1e-4 / 1e-5 / 1e-6 / 1e-7 / 1e-8 / 1e-9

## Внутренняя оценка векторов

Внутренняя оценка проводится по методу оценки сходства слов на размечанном корпусе 380 пар для измерения близости элементов спавочника. 

Элемент справочника $el = \{w_{1}, w_{2}, ..., w_{k}\}$, тогда вектор элемента вычисляется по следующей формуле

$$
x_{el} = \frac{1}{k}\sum_{i=1}^{k}w_{i}
$$

Все пары однозначно определены, следовательно средний показатель косинусной близости должен стремиться к единице для всех примеров.

## Внешняя оценка векторов

Внешняя оценка проводится с помощью заранее обученного классификатора присвоения бухгалтерского счета. Во время тестирование намеренно не будет поизводится тонкая настройка классификатора и дообучение векторов, в связи с тем чтобы более наглядно продемонстрировать зависимость качетсва классификатора от векторов.

Оценивание классификатора проводится по F1-score методу